# 🔍 Multimodal Retrieval System
## Using Danish's Smart Gallery Models & Pre-computed Embeddings

**Models:**
- Face: InsightFace buffalo_l (512-dim)
- Image: CLIP ViT-L-14 (768-dim)
- Detection: Your fine-tuned YOLOv8x

## 1️⃣ Setup

In [1]:
!nvidia-smi
import torch
print(f"CUDA: {torch.cuda.is_available()}")

Sat Jan 17 20:27:06 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip install -q insightface onnxruntime-gpu open-clip-torch faiss-cpu ultralytics
!pip install -q fastapi uvicorn python-multipart pyngrok nest-asyncio
print("✅ Dependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
✅ Dependencies installed!


In [3]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

Mounted at /content/drive
✅ Google Drive mounted!


In [4]:
# Re-download CIFAR-100 and extract to the same path
import os
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Create directory
os.makedirs("/content/datasets/images/cifar100", exist_ok=True)

# Download CIFAR-100
print("Downloading CIFAR-100...")
dataset = torchvision.datasets.CIFAR100(
    root='/content/datasets',
    train=True,
    download=True
)

# Save images with same naming convention
print("Saving images...")
for idx in tqdm(range(len(dataset))):
    img, label = dataset[idx]
    filename = f"/content/datasets/images/cifar100/{idx:05d}_class{label}.png"
    img.save(filename)

print(f"✅ Saved {len(dataset)} images!")

100%|██████████| 169M/169M [00:05<00:00, 29.2MB/s]


Saving images...


100%|██████████| 50000/50000 [00:18<00:00, 2773.48it/s]

✅ Saved 50000 images!


## 2️⃣ Configure Paths

In [5]:
import os
from pathlib import Path

# =============================================================================
# UPDATE THESE PATHS TO MATCH YOUR GOOGLE DRIVE
# =============================================================================
ML_OUTPUTS_BASE = "/content/drive/MyDrive/ml_pipeline_outputs"

# Face embeddings
FACE_EMBEDDINGS_PATH = f"{ML_OUTPUTS_BASE}/face_embeddings/face_embeddings.npy"
FACE_METADATA_PATH = f"{ML_OUTPUTS_BASE}/face_embeddings/identity_mapping.json"

# Image embeddings
IMAGE_EMBEDDINGS_DIR = f"{ML_OUTPUTS_BASE}/face_embeddings/image_embeddings"
CLIP_EMBEDDINGS_PATH = f"{IMAGE_EMBEDDINGS_DIR}/image_embeddings_clip.npy"
IMAGE_METADATA_PATH = f"{IMAGE_EMBEDDINGS_DIR}/image_metadata.json"
IMAGE_PATHS_CSV = f"{IMAGE_EMBEDDINGS_DIR}/image_paths.csv"
FAISS_CLIP_PATH = f"{IMAGE_EMBEDDINGS_DIR}/faiss_clip.index"

# YOLO model
YOLO_MODEL_PATH = f"{ML_OUTPUTS_BASE}/yolo_training/yolov8x_coco/weights/best.pt"

# Verify paths
print("Checking paths...")
for name, path in [
    ("Face embeddings", FACE_EMBEDDINGS_PATH),
    ("CLIP embeddings", CLIP_EMBEDDINGS_PATH),
    ("Image metadata", IMAGE_METADATA_PATH),
    ("Image paths CSV", IMAGE_PATHS_CSV),
    ("FAISS index", FAISS_CLIP_PATH),
]:
    status = "✅" if os.path.exists(path) else "❌"
    print(f"  {status} {name}")

Checking paths...
  ✅ Face embeddings
  ✅ CLIP embeddings
  ✅ Image metadata
  ✅ Image paths CSV
  ✅ FAISS index


## 3️⃣ Load Embeddings & Metadata

In [6]:
import numpy as np
import json
import pandas as pd
import faiss

# Load face embeddings
print("Loading embeddings...")
face_embeddings = np.load(FACE_EMBEDDINGS_PATH) if os.path.exists(FACE_EMBEDDINGS_PATH) else None
if face_embeddings is not None:
    print(f"  ✅ Face: {face_embeddings.shape}")

# Load CLIP embeddings
clip_embeddings = np.load(CLIP_EMBEDDINGS_PATH) if os.path.exists(CLIP_EMBEDDINGS_PATH) else None
if clip_embeddings is not None:
    print(f"  ✅ CLIP: {clip_embeddings.shape}")

# Load image metadata (JSON)
image_metadata = {}
if os.path.exists(IMAGE_METADATA_PATH):
    with open(IMAGE_METADATA_PATH, 'r') as f:
        image_metadata = json.load(f)
    print(f"  ✅ Metadata: {len(image_metadata)} entries")

# Load image paths (CSV) - this has the actual file paths
image_paths_list = []
if os.path.exists(IMAGE_PATHS_CSV):
    df = pd.read_csv(IMAGE_PATHS_CSV)
    image_paths_list = df['path'].tolist() if 'path' in df.columns else df.iloc[:, 0].tolist()
    print(f"  ✅ Image paths: {len(image_paths_list)} files")

# Load face metadata
face_metadata = {}
if os.path.exists(FACE_METADATA_PATH):
    with open(FACE_METADATA_PATH, 'r') as f:
        face_metadata = json.load(f)
    print(f"  ✅ Face metadata: {len(face_metadata)} entries")

# Load or build FAISS index
print("\nLoading FAISS index...")
if os.path.exists(FAISS_CLIP_PATH):
    image_index = faiss.read_index(FAISS_CLIP_PATH)
    print(f"  ✅ Loaded pre-built index: {image_index.ntotal} vectors")
elif clip_embeddings is not None:
    image_index = faiss.IndexFlatIP(768)
    clip_norm = clip_embeddings / np.linalg.norm(clip_embeddings, axis=1, keepdims=True)
    image_index.add(clip_norm.astype(np.float32))
    print(f"  ✅ Built new index: {image_index.ntotal} vectors")
else:
    image_index = faiss.IndexFlatIP(768)

# Build face index
if face_embeddings is not None:
    face_index = faiss.IndexFlatIP(512)
    face_norm = face_embeddings / np.linalg.norm(face_embeddings, axis=1, keepdims=True)
    face_index.add(face_norm.astype(np.float32))
    print(f"  ✅ Face index: {face_index.ntotal} vectors")
else:
    face_index = faiss.IndexFlatIP(512)

print(f"\n📊 Total: {image_index.ntotal} images, {face_index.ntotal} faces")

Loading embeddings...
  ✅ Face: (9962, 512)
  ✅ CLIP: (4998, 768)
  ✅ Metadata: 11 entries
  ✅ Image paths: 4998 files
  ✅ Face metadata: 9962 entries

Loading FAISS index...
  ✅ Loaded pre-built index: 4998 vectors
  ✅ Face index: 9962 vectors

📊 Total: 4998 images, 9962 faces


## 4️⃣ Load Models

In [7]:
import torch
from insightface.app import FaceAnalysis
import open_clip
from PIL import Image
import io

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# InsightFace
print("Loading InsightFace...")
face_analyzer = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
face_analyzer.prepare(ctx_id=0, det_size=(640, 640))
print("✅ InsightFace loaded")

# CLIP ViT-L-14
print("Loading CLIP ViT-L-14...")
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai', device=DEVICE)
clip_model.eval()
clip_tokenizer = open_clip.get_tokenizer('ViT-L-14')
print("✅ CLIP loaded")

# YOLO (optional)
yolo_model = None
if os.path.exists(YOLO_MODEL_PATH):
    from ultralytics import YOLO
    yolo_model = YOLO(YOLO_MODEL_PATH)
    print("✅ YOLO loaded")

print("\n🎉 All models ready!")

Loading InsightFace...
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 110945.48KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


✅ CLIP loaded
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ YOLO loaded

🎉 All models ready!


## 5️⃣ Helper Functions

In [8]:
import numpy as np
from PIL import Image
import io

def get_image_path(idx):
    """Get image path from index."""
    if idx < len(image_paths_list):
        return image_paths_list[idx]
    return None

def get_image_metadata(idx):
    """Get metadata for image index."""
    # Try different key formats
    for key in [str(idx), idx, f"image_{idx}"]:
        if key in image_metadata:
            return image_metadata[key]
    # Return basic info if no metadata found
    path = get_image_path(idx)
    return {"path": path, "index": idx} if path else {"index": idx}

def embed_face(image_bytes):
    """Generate 512-dim face embedding."""
    image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    img_array = np.array(image)[:, :, ::-1]
    faces = face_analyzer.get(img_array)
    if not faces:
        return None
    best_face = max(faces, key=lambda f: f.det_score)
    if best_face.det_score < 0.4:
        return None
    emb = best_face.embedding
    return (emb / np.linalg.norm(emb)).astype(np.float32)

def embed_image(image_bytes):
    """Generate 768-dim CLIP image embedding."""
    image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    img_input = clip_preprocess(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        features = clip_model.encode_image(img_input)
    emb = features.cpu().numpy().flatten()
    return (emb / np.linalg.norm(emb)).astype(np.float32)

def embed_text(text):
    """Generate 768-dim CLIP text embedding."""
    tokens = clip_tokenizer([text]).to(DEVICE)
    with torch.no_grad():
        features = clip_model.encode_text(tokens)
    emb = features.cpu().numpy().flatten()
    return (emb / np.linalg.norm(emb)).astype(np.float32)

print("✅ Helper functions defined")
print(f"   Image paths available: {len(image_paths_list)}")

✅ Helper functions defined
   Image paths available: 4998


## 6️⃣ Search Functions (WITH Image Paths)

In [9]:
def search_by_face(image_bytes, top_k=10):
    """Search by face - returns results with image paths."""
    face_emb = embed_face(image_bytes)
    if face_emb is None:
        raise ValueError("No face detected")

    scores, indices = face_index.search(face_emb.reshape(1, -1), top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0:
            meta = get_image_metadata(int(idx))
            img_path = get_image_path(int(idx))
            results.append({
                'index': int(idx),
                'score': float(score),
                'type': 'face',
                'image_path': img_path,
                'metadata': meta
            })
    return results


def search_by_image(image_bytes, top_k=10):
    """Search by image - returns results with image paths."""
    image_emb = embed_image(image_bytes)

    scores, indices = image_index.search(image_emb.reshape(1, -1), top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0:
            meta = get_image_metadata(int(idx))
            img_path = get_image_path(int(idx))
            results.append({
                'index': int(idx),
                'score': float(score),
                'type': 'image',
                'image_path': img_path,
                'metadata': meta
            })
    return results


def search_by_text(query, top_k=10):
    """Search by text - returns results with image paths."""
    text_emb = embed_text(query)

    scores, indices = image_index.search(text_emb.reshape(1, -1), top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0:
            meta = get_image_metadata(int(idx))
            img_path = get_image_path(int(idx))
            results.append({
                'index': int(idx),
                'score': float(score),
                'type': 'text',
                'image_path': img_path,
                'metadata': meta
            })
    return results

print("✅ Search functions ready (with image paths)")

✅ Search functions ready (with image paths)


## 7️⃣ Test Search

In [20]:
# Test search
if image_index.ntotal > 0:
    results = search_by_text("Cat", top_k=5)
    print("🔍 Results for 'person ':")
    for i, r in enumerate(results, 1):
        print(f"  {i}. Score: {r['score']:.3f} | Path: {r['image_path']}")

🔍 Results for 'person ':
  1. Score: 0.220 | Path: /content/datasets/images/cifar100/00724_class66.png
  2. Score: 0.218 | Path: /content/datasets/images/cifar100/00916_class4.png
  3. Score: 0.217 | Path: /content/datasets/images/cifar100/02305_class64.png
  4. Score: 0.216 | Path: /content/datasets/images/cifar100/03322_class32.png
  5. Score: 0.215 | Path: /content/datasets/images/cifar100/03989_class66.png


## 8️⃣ FastAPI Server

In [21]:
%%writefile server.py
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, Response
import time
import os

app = FastAPI(title="Multimodal Retrieval API")

# CORS middleware for all routes
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"]
)

@app.get("/")
async def root():
    from __main__ import image_index, face_index, image_paths_list
    return {
        "status": "healthy",
        "images_indexed": image_index.ntotal,
        "faces_indexed": face_index.ntotal,
        "image_paths_loaded": len(image_paths_list)
    }

@app.post("/search/face")
async def api_search_face(file: UploadFile = File(...), top_k: int = Form(default=10)):
    from __main__ import search_by_face
    start = time.time()
    contents = await file.read()
    try:
        results = search_by_face(contents, top_k)
    except ValueError as e:
        raise HTTPException(400, str(e))
    return {"query_type": "face", "results": results, "search_time_ms": round((time.time()-start)*1000, 2)}

@app.post("/search/image")
async def api_search_image(file: UploadFile = File(...), top_k: int = Form(default=10)):
    from __main__ import search_by_image
    start = time.time()
    contents = await file.read()
    results = search_by_image(contents, top_k)
    return {"query_type": "image", "results": results, "search_time_ms": round((time.time()-start)*1000, 2)}

@app.post("/search/text")
async def api_search_text(query: str = Form(...), top_k: int = Form(default=10)):
    from __main__ import search_by_text
    start = time.time()
    results = search_by_text(query, top_k)
    return {"query_type": "text", "results": results, "search_time_ms": round((time.time()-start)*1000, 2)}

@app.get("/image/{idx}")
async def get_image(idx: int):
    """Serve image by index with CORS headers."""
    from __main__ import get_image_path
    path = get_image_path(idx)
    if path and os.path.exists(path):
        # Read file and return with CORS headers
        with open(path, "rb") as f:
            content = f.read()

        # Detect content type
        ext = path.lower().split('.')[-1]
        content_types = {
            'jpg': 'image/jpeg',
            'jpeg': 'image/jpeg',
            'png': 'image/png',
            'gif': 'image/gif',
            'webp': 'image/webp'
        }
        content_type = content_types.get(ext, 'image/jpeg')

        return Response(
            content=content,
            media_type=content_type,
            headers={
                "Access-Control-Allow-Origin": "*",
                "Cache-Control": "public, max-age=3600"
            }
        )
    raise HTTPException(404, "Image not found")

@app.post("/embed/face")
async def api_embed_face(file: UploadFile = File(...)):
    from __main__ import embed_face
    contents = await file.read()
    emb = embed_face(contents)
    if emb is None:
        raise HTTPException(400, "No face detected")
    return {"embedding": emb.tolist(), "dimension": 512}

@app.post("/embed/image")
async def api_embed_image(file: UploadFile = File(...)):
    from __main__ import embed_image
    contents = await file.read()
    return {"embedding": embed_image(contents).tolist(), "dimension": 768}

@app.post("/embed/text")
async def api_embed_text(query: str = Form(...)):
    from __main__ import embed_text
    return {"embedding": embed_text(query).tolist(), "dimension": 768}

Writing server.py


## 9️⃣ Run Server with ngrok

In [22]:
# Set your ngrok auth token
!ngrok authtoken 38FY2ZkXKAJzHfsWULq6acTAtpj_5sc8zM3Yg9BobtgdiCzbc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from pyngrok import ngrok
import asyncio

# Start ngrok
public_url = ngrok.connect(8000)
print(f"\n🌐 PUBLIC URL: {public_url.public_url}")

# IMPORTANT: Reload the server module
import importlib
import server
importlib.reload(server)

from server import app
import uvicorn

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server_instance = uvicorn.Server(config)

print("\n✅ Server starting...")
await server_instance.serve()


🌐 PUBLIC URL: https://insensible-gregory-alphanumerically.ngrok-free.dev


INFO:     Started server process [5093]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



✅ Server starting...
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET / HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "POST /search/face HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/3343 HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/4826 HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/93 HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/1395 HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/374 HTTP/1.1" 200 OK
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/7638 HTTP/1.1" 404 Not Found
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/5405 HTTP/1.1" 404 Not Found
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/7668 HTTP/1.1" 404 Not Found
INFO:     2a00:23ee:1698:3122:659a:23f6:d26e:75ae:0 - "GET /image/1878 HTTP/1.1" 200 OK
